**Step 1 **

In [ ]:
%%capture

!pip install openai pymupdf tqdm chromadb spacy sentence-transformers
!python -m spacy download  el_core_news_sm
!python -m spacy download en_core_web_sm

**DATA INGESTION **

In [ ]:
import re
import os
import urllib.request
import uuid
import shutil
import fitz
from tqdm.autonotebook import tqdm

<ipython-input-4-64c3754408ad>:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
def download_pdf(url, output_path):
    os.makedirs(output_path, exist_ok=True)
    local_pdf = f"{output_path}/{uuid.uuid4().hex}.pdf"
    if url == output_path:
        return
    try:
        urllib.request.urlretrieve(url, local_pdf)
    except ValueError:
        shutil.copy(url, local_pdf)
    return local_pdf


def preprocess(text):
    text = text.replace("-\n", "")  # no word breaks
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\.+", ".", text)
    return text


def pdf2text(path, start_page=0, end_page=-1):
    print("Parsing PDF")
    doc = fitz.open(path)
    total_pages = doc.page_count
    print(f"PDF contains {total_pages} pages")
    if end_page <= 0:
        end_page = total_pages

    text_list = []
    for i in tqdm(
        range(start_page, end_page),
        desc=f"Converting PDF to text. Pages: {start_page}-{end_page}",
    ):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)
    doc.close()
    return " ".join(text_list)

In [ ]:
# Uncomment for a greek example
input_pdf_greek = "http://ebooks.edu.gr/ebooks/v/pdf/8547/2488/22-0047-02_Biologia_G-Lykeiou-ThSp-SpYg_Vivlio-Mathiti/"
local_pdf_greek = download_pdf(input_pdf_greek, "data")


text_greek = pdf2text(local_pdf_greek, start_page=0, end_page=-1)
print(text_greek)

Parsing PDF
PDF contains 182 pages


Converting PDF to text. Pages: 0-182:   0%|          | 0/182 [00:00<?, ?it/s]

 Γ΄ ΓΕΝΙΚΟΥ ΛΥΚΕΙΟΥ ΙΝΣΤΙΤΟΥΤΟ ΤΕΧΝΟΛΟΓΙΑΣ ΥΠΟΛΟΓΙΣΤΩΝ ΚΑΙ ΕΚΔΟΣΕΩN «ΔΙΟΦΑΝΤΟΣ» Ομάδας Προσανατολισμού θετικών Σπουδών & Σπουδών Υγείας ΒΙΟΛΟΓΙΑ ΥΠΟΥΡΓΕΙΟ ΠΑΙΔΕΙΑΣ ΚΑΙ ΘΡΗΣΚΕΥΜΑΤΩΝ ΙΝΣΤΙΤΟΥΤΟ ΕΚΠΑΙΔΕΥΤΙΚΗΣ ΠΟΛΙΤΙΚΗΣ ΤΕΥΧΟΣ Α΄  ΒΙΟΛΟΓΙΑ Ομάδας Προσανατολισμού Θετικών Σπουδών & Σπουδών Υγείας Γ´ ΓΕΝΙΚΟΥ ΛΥΚΕΙΟΥ Tεύχος Α´  ΟΜΑΔΑ ΣΥΓΓΡΑΦΗΣ ΑΘΑΝΑΣΙΟΣ ΚΑΨΑΛΗΣ, βιολόγος, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΙΩΑΝΝΗΣ-ΕΥΑΓΓΕΛΟΣ ΜΠΟΥΡΜΠΟΥΧΑΚΗΣ, βιολόγος, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΒΑΣΙΛΙΚΗ ΠΕΡΑΚΗ, δρ Βιολογίας, μον. πάρεδρος Παιδαγωγικού Ινστιτούτου. ΣΤΕΡΓΙΟΣ ΣΑΛΑΜΑΣΤΡΑΚΗΣ, Msc Ωκεανογραφίας, δρ Βιολογίας, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΥΠΕΥΘΥΝΗ ΣΤΟ ΠΛΑΙΣΙΟ ΤΟΥ ΠΑΙΔΑΓΩΓΙΚΟΥ ΙΝΣΤΙΤΟΥΤΟΥ ΒΑΣΙΛΙΚΗ ΠΕΡΑΚΗ, δρ Βιολογίας, μον. πάρεδρος Παιδαγωγικού Ινστιτούτου. ΓΛΩΣΣΙΚΗ ΕΠΙΜΕΛΕΙΑ ΚΛΕΙΔΩΝΑΡΗ ΜΑΙΡΙΤΑ, φιλόλογος, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΣΧΕΔΙΑΣΜΟΣ ΕΝΤΥΠΟΥ ΚΑΙ ΚΑΛΛΙΤΕΧΝΙΚΗ ΕΠΙΜΕΛΕΙΑ ΣΠΥΡΟΣ I. ΠΑΠΑΣΠΥΡΟΥ, καθηγητής Εφαρμογών του ΤΕΙ Ηπείρου. ΟΜΑΔΑ ΚΡΙΣΗΣ ΑΡΝΑΟΥΤΑΚΗΣ ΙΩΑΝΝΗΣ, σχολικός σύ

In [ ]:
text_greek


" Γ΄ ΓΕΝΙΚΟΥ ΛΥΚΕΙΟΥ ΙΝΣΤΙΤΟΥΤΟ ΤΕΧΝΟΛΟΓΙΑΣ ΥΠΟΛΟΓΙΣΤΩΝ ΚΑΙ ΕΚΔΟΣΕΩN «ΔΙΟΦΑΝΤΟΣ» Ομάδας Προσανατολισμού θετικών Σπουδών & Σπουδών Υγείας ΒΙΟΛΟΓΙΑ ΥΠΟΥΡΓΕΙΟ ΠΑΙΔΕΙΑΣ ΚΑΙ ΘΡΗΣΚΕΥΜΑΤΩΝ ΙΝΣΤΙΤΟΥΤΟ ΕΚΠΑΙΔΕΥΤΙΚΗΣ ΠΟΛΙΤΙΚΗΣ ΤΕΥΧΟΣ Α΄  ΒΙΟΛΟΓΙΑ Ομάδας Προσανατολισμού Θετικών Σπουδών & Σπουδών Υγείας Γ´ ΓΕΝΙΚΟΥ ΛΥΚΕΙΟΥ Tεύχος Α´  ΟΜΑΔΑ ΣΥΓΓΡΑΦΗΣ ΑΘΑΝΑΣΙΟΣ ΚΑΨΑΛΗΣ, βιολόγος, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΙΩΑΝΝΗΣ-ΕΥΑΓΓΕΛΟΣ ΜΠΟΥΡΜΠΟΥΧΑΚΗΣ, βιολόγος, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΒΑΣΙΛΙΚΗ ΠΕΡΑΚΗ, δρ Βιολογίας, μον. πάρεδρος Παιδαγωγικού Ινστιτούτου. ΣΤΕΡΓΙΟΣ ΣΑΛΑΜΑΣΤΡΑΚΗΣ, Msc Ωκεανογραφίας, δρ Βιολογίας, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΥΠΕΥΘΥΝΗ ΣΤΟ ΠΛΑΙΣΙΟ ΤΟΥ ΠΑΙΔΑΓΩΓΙΚΟΥ ΙΝΣΤΙΤΟΥΤΟΥ ΒΑΣΙΛΙΚΗ ΠΕΡΑΚΗ, δρ Βιολογίας, μον. πάρεδρος Παιδαγωγικού Ινστιτούτου. ΓΛΩΣΣΙΚΗ ΕΠΙΜΕΛΕΙΑ ΚΛΕΙΔΩΝΑΡΗ ΜΑΙΡΙΤΑ, φιλόλογος, εκπαιδευτικός Δ/θμιας Εκπαίδευσης. ΣΧΕΔΙΑΣΜΟΣ ΕΝΤΥΠΟΥ ΚΑΙ ΚΑΛΛΙΤΕΧΝΙΚΗ ΕΠΙΜΕΛΕΙΑ ΣΠΥΡΟΣ I. ΠΑΠΑΣΠΥΡΟΥ, καθηγητής Εφαρμογών του ΤΕΙ Ηπείρου. ΟΜΑΔΑ ΚΡΙΣΗΣ ΑΡΝΑΟΥΤΑΚΗΣ ΙΩΑΝΝΗΣ, σχολικός σ

# Evaluation

In [ ]:
import os
import subprocess

In [ ]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/1

In [ ]:
import json
from ragas.metrics import answer_relevancy, faithfulness

def evaluate_questions(json_file_path, retriever, generator):
    """Evaluates a JSON file of questions using RAGAS.

    Args:
        json_file_path: The path to the JSON file containing the questions.
        retriever: The retriever to use for retrieving relevant documents.
        generator: The generator to use for generating answers.

    Returns:
        A dictionary containing the evaluation results.
    """

    # Load the questions from the JSON file
    with open(json_file_path, 'r') as f:
        questions = json.load(f)

    results = []

    # Evaluate each question
    for question in tqdm(questions, desc="Evaluating questions"):
        context = retriever.retrieve(question['question'])
        answer = generator.generate(question['question'], context)

        # Calculate metrics
        relevancy = answer_relevancy(question['question'], answer, context)
        faithfulness = faithfulness(question['question'], answer, context)

        results.append({
            "question": question['question'],
            "answer": answer,
            "relevancy": relevancy,
            "faithfulness": faithfulness,
        })

    return results

# **CHUNKING**

In [ ]:
import spacy

nlp_greek = spacy.load("el_core_news_sm")

In [ ]:
def chunk_text(text, spacy_nlp=nlp_greek, chunk_size=128):
    print("Split document into sentences using spacy")
    sentences = [sent.text for sent in nlp_greek(text).sents]
    print(f"Document contains #{len(sentences)} sentences")

    print(f"Merge consecutive sentences up to chunk size = {chunk_size}")
    chunks = [sentences[0]]
    for index in tqdm(range(1, len(sentences)), desc="Chunking text"):
        potential_chunk = "\n".join([chunks[-1], sentences[index]])
        num_tokens_in_chunk = len([token for token in spacy_nlp(potential_chunk)])
        if num_tokens_in_chunk < chunk_size:
            chunks[-1] = potential_chunk
        else:
            chunks.append(sentences[index])

    print(f"Created #{len(chunks)} chunks")
    return chunks

In [ ]:
chunks_greek = chunk_text(text_greek, spacy_nlp=nlp_greek, chunk_size=128)

Split document into sentences using spacy
Document contains #3327 sentences
Merge consecutive sentences up to chunk size = 128


Chunking text:   0%|          | 0/3326 [00:00<?, ?it/s]

Created #520 chunks


In [ ]:
print(chunks_greek[8])

Η απλή περιγραφή των οργανισμών και των φαινομένων που τους αφορούν έδωσε τη θέση της στον πειραματισμό και την ανακάλυψη.
Αυτό επέτρεψε την κατανόηση των δομών και των διαδικασιών της ζωής, καθώς και της σχέσης που υπάρχει μεταξύ τους.
Με το βιβλίο που κρατάς στα χέρια σου, ελπίζουμε να σε βοηθήσουμε να προσεγγίσεις το βασικότερο ίσως από τα αντικείμενα μελέτης της Βιολογίας: τη δομική και λειτουργική μονάδα όλων των οργανισμών, το κύτταρο.


In [ ]:
import chromadb
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings

In [ ]:
class MultilingualSentenceTransformer(EmbeddingFunction):
    """ Create a ChromaDB embedding function based on the chosen model """
    def __init__(
        self,
        # model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # 384d.
        model="BAAI/bge-m3",  # 1024d.
    ):
        self.model = model
        self.embedder = self.initialize_model(model)

    def initialize_model(self, model):
        from sentence_transformers import SentenceTransformer

        embedder = SentenceTransformer(model, device="cuda")
        return embedder

    def __call__(self, sentences: Documents) -> Embeddings:
        return self.embedder.encode(sentences, convert_to_numpy=True).tolist()  # type: ignore


def batchify(iterable, n=1):
    """ For efficient embedding extraction """
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx : min(ndx + n, l)]


def create_collection(chunks, collection_name="rag_from_scratch", batch_size=12):
    print("Create and index vector store")

    client = chromadb.EphemeralClient()  # DB is created in memory. Use persistent client for a persistent DB
    # client = chromadb.PersistentClient("./chroma")
    embedding_function = MultilingualSentenceTransformer()
    collection = client.create_collection(
        collection_name,
        metadata={"hnsw:space": "cosine"},
        embedding_function=embedding_function,
    )

    id_num = 0
    for batch in tqdm(
        batchify(chunks, n=batch_size),
        desc="Indexing vector database",
        total=len(chunks) // batch_size,
    ):
        texts = batch
        collection.add(
            documents=texts,  # type: ignore
            ids=[f"id{idx}" for idx in range(id_num, id_num + len(texts))],
        )
        id_num += len(texts)

    return collection

In [ ]:
COLLECTION_NAME_GREEK = "rag_from_scratch_greek3"
collection_greek = create_collection(chunks_greek, collection_name=COLLECTION_NAME_GREEK)

Create and index vector store


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Indexing vector database:   0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
import pprint

In [ ]:
def retrieve_relevant_passages(query, chroma_collection=collection_greek, n_results=4):
    result = chroma_collection.query(
        query_texts="What is MLM?",
        n_results=4,
    )
    return result["documents"][0]

In [ ]:
passages_greek = retrieve_relevant_passages("Τι είναι η ομάδα αίματος", chroma_collection=collection_greek)

for passage in passages_greek:
    print(passage)
    print("--------------------------------------")


Σ' αυτά ανήκουν οι αμυλοπλάστες, οι χλωροπλάστες κ.ά. Πολυμερές:
Μακρομόριο, που αποτελείται από όμοιες υπομονάδες (μονομερή).
Πολυπεπτιδική αλυσίδα:
Πολυμερές από αμινοξέα.
Πουρίνες: Οργανικά μόρια, παράγωγα των οποίων είναι οι αζωτούχες βάσεις αδενίνη και γουανίνη.
Προκαρυωτικό κύτταρο:
Το κύτταρο που δε διαθέτει σχηματισμούς οι οποίοι να αποτελούνται από στοιχειώδη μεμβράνη (εκτός της πλασματικής).
Πρόφαση:
Το πρώτο στάδιο της μειωτικής διαίρεσης, κατά το οποίο εμφανίζονται τα χρωμοσώματα και αρχίζει να σχηματίζεται η άτρακτος.
Πρόφαση I: Πρόφαση της πρώτης μειωτικής διαίρεσης.
Σ' αυτή γίνεται σύναψη των ομόλογων χρωμοσωμάτων και επιχιασμός.
--------------------------------------
Εφυμενίδα: Ένα συνεχές αδιάβροχο υμένιο, που καλύπτει εξωτερικά την επιδερμίδα και του οποίου το πάχος κυμαίνεται ανάλογα με τις συνθήκες του περιβάλλοντος.
ζ Ζύμες: Μονοκύτταροι μικροοργανισμοί, που ανήκουν στην κατηγορία των μυκήτων και φέρουν σε πέρας την αλκοολική ζύμωση.
Ζύμωση: Μεταβολική πορεία, κατά

In [ ]:
PROMPT_TEMPLATE = """Passages:\n\n
{passages}\n\n
Query: {query}"""


def format_prompt(query, collection_greek, n_neighbors=3):
    results = collection_greek.query(query_texts=query, n_results=n_neighbors)

    if not results["documents"]:
        print("No relevant documents found. Returning original query")
        return query

    passages = "\n\n".join(results["documents"][0])

    prompt = PROMPT_TEMPLATE.format(passages=passages, query=query)

    return prompt


test_prompt = format_prompt("What is MLM;", collection_greek)

print(test_prompt)

Passages:


Σ' αυτά ανήκουν οι αμυλοπλάστες, οι χλωροπλάστες κ.ά. Πολυμερές:
Μακρομόριο, που αποτελείται από όμοιες υπομονάδες (μονομερή).
Πολυπεπτιδική αλυσίδα:
Πολυμερές από αμινοξέα.
Πουρίνες: Οργανικά μόρια, παράγωγα των οποίων είναι οι αζωτούχες βάσεις αδενίνη και γουανίνη.
Προκαρυωτικό κύτταρο:
Το κύτταρο που δε διαθέτει σχηματισμούς οι οποίοι να αποτελούνται από στοιχειώδη μεμβράνη (εκτός της πλασματικής).
Πρόφαση:
Το πρώτο στάδιο της μειωτικής διαίρεσης, κατά το οποίο εμφανίζονται τα χρωμοσώματα και αρχίζει να σχηματίζεται η άτρακτος.
Πρόφαση I: Πρόφαση της πρώτης μειωτικής διαίρεσης.
Σ' αυτή γίνεται σύναψη των ομόλογων χρωμοσωμάτων και επιχιασμός.

Εφυμενίδα: Ένα συνεχές αδιάβροχο υμένιο, που καλύπτει εξωτερικά την επιδερμίδα και του οποίου το πάχος κυμαίνεται ανάλογα με τις συνθήκες του περιβάλλοντος.
ζ Ζύμες: Μονοκύτταροι μικροοργανισμοί, που ανήκουν στην κατηγορία των μυκήτων και φέρουν σε πέρας την αλκοολική ζύμωση.
Ζύμωση: Μεταβολική πορεία, κατά την οποία παράγεται ΑΤΡ α

In [ ]:
test_prompt_greek = format_prompt("What is MLM;", collection_greek)

print(test_prompt_greek)

Passages:


Σ' αυτά ανήκουν οι αμυλοπλάστες, οι χλωροπλάστες κ.ά. Πολυμερές:
Μακρομόριο, που αποτελείται από όμοιες υπομονάδες (μονομερή).
Πολυπεπτιδική αλυσίδα:
Πολυμερές από αμινοξέα.
Πουρίνες: Οργανικά μόρια, παράγωγα των οποίων είναι οι αζωτούχες βάσεις αδενίνη και γουανίνη.
Προκαρυωτικό κύτταρο:
Το κύτταρο που δε διαθέτει σχηματισμούς οι οποίοι να αποτελούνται από στοιχειώδη μεμβράνη (εκτός της πλασματικής).
Πρόφαση:
Το πρώτο στάδιο της μειωτικής διαίρεσης, κατά το οποίο εμφανίζονται τα χρωμοσώματα και αρχίζει να σχηματίζεται η άτρακτος.
Πρόφαση I: Πρόφαση της πρώτης μειωτικής διαίρεσης.
Σ' αυτή γίνεται σύναψη των ομόλογων χρωμοσωμάτων και επιχιασμός.

Εφυμενίδα: Ένα συνεχές αδιάβροχο υμένιο, που καλύπτει εξωτερικά την επιδερμίδα και του οποίου το πάχος κυμαίνεται ανάλογα με τις συνθήκες του περιβάλλοντος.
ζ Ζύμες: Μονοκύτταροι μικροοργανισμοί, που ανήκουν στην κατηγορία των μυκήτων και φέρουν σε πέρας την αλκοολική ζύμωση.
Ζύμωση: Μεταβολική πορεία, κατά την οποία παράγεται ΑΤΡ α

In [ ]:
import openai
from openai import OpenAI

# For Meltemi
MELTEMI_API_KEY = "sk-RYF0g_hDDIa2TLiHFboZ1Q"
MELTEMI_BASE_URL = "http://ec2-3-19-37-251.us-east-2.compute.amazonaws.com:4000/"
MELTEMI_CLIENT = OpenAI(api_key=MELTEMI_API_KEY, base_url=MELTEMI_BASE_URL)

In [ ]:
RAG_SYSTEM_PROMPT = """Instructions: Compose a comprehensive reply to the query, based on the provided passages.
If the search results mention multiple subjects with the same name, create separate answers for each.
In your answers, maintain the appropriate knowledge level for a person that is consuming this material.
You MAY include appropriate extra information for providing clear, simple and comprehensive clarifications,
if the user explicitly asks for it, else you only respond based on the provided passages.
Do not make the responses neither too advanced, nor too simplistic.
If the text does not relate at all to the query, propose more appropriate prompt queries based on the book contents.
Provide your answer in the same language as the user's query."""

PLAIN_SYSTEM_PROMPT = """Instructions: Answer the user's questions to the best of your knowledge."""


def query_llm(query, system_prompt, model="gpt"):
    if "gpt" in model:
        model_name = "gpt-3.5-turbo"
        client = GPT_CLIENT
    else:
        model_name = "meltemi"
        client = MELTEMI_CLIENT
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
    )
    return response.choices[0].message.content, response.usage



def plain_query(query, model="gpt"):
    print("---------------------------------------------------------------")
    print("Plain query without using  RAG")
    print("---------------------------------------------------------------")

    prompt = query
    response, usage = query_llm(prompt, PLAIN_SYSTEM_PROMPT, model=model)
    print()
    print("QUERY:")
    print(prompt)
    print("---------------------------------------------------------------")
    print()
    print("LLM RESPONSE:")
    print(response)
    print("---------------------------------------------------------------")
    print()
    print("TOKEN USAGE:")
    print(usage)
    print("---------------------------------------------------------------")


def rag_query(query, model="gpt", collection=collection_greek):
    print("---------------------------------------------------------------")
    print("Query using  RAG")
    print("---------------------------------------------------------------")

    prompt = format_prompt(query, collection)
    response, usage = query_llm(prompt, RAG_SYSTEM_PROMPT, model=model)
    print()
    print("QUERY:")
    print(prompt)
    print("---------------------------------------------------------------")
    print()
    print("LLM RESPONSE:")
    print(response)
    print("---------------------------------------------------------------")
    print()
    print("TOKEN USAGE:")
    print(usage)
    print("---------------------------------------------------------------")

In [ ]:
QUERY1_GREEK = "Τι ξέρεις για τις ομάδες αίματος;"
QUERY2_GREEK = "Πόσα γκολ έχει βάλει ο Λιονέλ Μέσι;"

In [ ]:
plain_query(QUERY1_GREEK, model="meltemi")

---------------------------------------------------------------
Plain query without using  RAG
---------------------------------------------------------------

QUERY:
Τι ξέρεις για τις ομάδες αίματος;
---------------------------------------------------------------

LLM RESPONSE:
Οι ομάδες αίματος αναφέρονται σε μία κατάταξη ανθρώπινων ερυθρών αιμοσφαιρίων βασισμένη στα επίπεδα των πρωτεϊνών επιφανειών που ονομάζονται αντιγόνα. Αυτά τα αντιγόνα καθορίζουν τις αλληλεπιδράσεις μεταξύ των ερυθρών αιμοσφαιρίων και του πλάσματος του αίματος. Υπάρχουν τέσσερις κύριοι τύποι ομάδων αίματος: Α, Β, ΑΒ και Ο, αναφερόμενοι ως τα αντιγόνα Α και Β που είναι παρόντα ή απόντα στην επιφάνεια των ερυθρών αιμοσφαιρίων. Τα αντιγόνα Α και Β είναι αντιγόνα ισοτόπων του συστήματος ΑΒΟ και το Α δηλώνειτην
---------------------------------------------------------------

TOKEN USAGE:
CompletionUsage(completion_tokens=100, prompt_tokens=59, total_tokens=159, completion_tokens_details=None, prompt_tokens_details=N

In [ ]:
plain_query(QUERY2_GREEK, model="meltemi")

---------------------------------------------------------------
Plain query without using  RAG
---------------------------------------------------------------

QUERY:
Πόσα γκολ έχει βάλει ο Λιονέλ Μέσι;
---------------------------------------------------------------

LLM RESPONSE:
Ο Λιονέλ Μέσι είναι ένας διάσημος ποδοσφαιριστής και έχει παίξει για την Μπαρτσελόνα και την Εθνική Αργεντινής για μεγάλο διάστημα. Έχει παίξει σημαντικό ρόλο στις ποδοσφαιρικές του επιτυχίες και έχει πετύχει πολλά γκολ στα χρόνια της καριέρας του. Ωστόσο, χωρίς συγκεκριμένες πληροφορίες για τον ακριβή αριθμό γκολ που έχει πετύχει ή το έτος, θα έκανα μια λίστα με τους τίτλους του:

* Τέσσερα Κόσμοι ΦΙΦΑ (2009, 2014, 2015, 2012)
* Τρία Παγκόσμια Κύπελλα ΦΙΦΑ
---------------------------------------------------------------

TOKEN USAGE:
CompletionUsage(completion_tokens=100, prompt_tokens=63, total_tokens=163, completion_tokens_details=None, prompt_tokens_details=None)
-------------------------------------------